<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=77aa1bdad9eb5ecf3e265e061321ec6c2405ffd51529e4a36cd8cf0324c67cf7
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-15 13:12:49
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.41 C
-------------------
Today PnL: 27.36 K (0.19%)
Current PnL: -26.19 L (-16.92%)
CY Booked + Current PnL: -11.90 L (-7.69%)
-------------------
Total profit:  1.43 L
Total loss:  -27.62 L
-------------------
Total Booked + Current PnL: 15.08 L (11.8%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.14%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.82 L (64.48%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.42%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.17,-16.59,20.00,0.09,16811.0,-16721.0,84056.0,95.09,44.0,M-SC,2.37,253.0,-0.99,0.60,10.43,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,1.03,-11.45,12.95,0.02,20018.0,-19981.0,154578.0,135.49,55.0,M-SC,15.39,234.0,-1.00,1.10,34.46,OX40N,NTT,PAINTS
82,VOLTAS,1530.00,0.49,8.27,10.55,19.69,21902.0,15858.0,207600.0,-2.32,56.0,X-MC,2.56,78.0,0.72,1.48,15.63,XY25,NTT,AC
50,MASFIN,398.61,0.59,-0.51,22.67,22.05,22100.0,-495.0,97485.0,-14.18,59.0,H-SC,7.18,164.0,-0.02,0.70,41.70,XR,ATH,FINANCE
43,ITC,452.00,0.47,-1.47,12.44,10.78,29455.0,-3540.0,236778.0,-41.15,46.0,X-LC,1.91,5.0,-0.12,1.69,3.69,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,9.91,-39.45,123.40,35.27,84844.0,-44794.0,68755.0,-53.72,50.0,L-SC,10.88,270.0,-0.53,0.49,68.28,XR,NTT,HOTELS
57,RAJOOENG,143.10,7.03,-33.73,110.66,39.61,75171.0,-34570.0,67930.0,-55.26,38.0,H-SC,28.03,135.0,-0.46,0.48,7.03,AR,ATH,MISC
44,JCHAC,2282.00,3.54,-35.24,54.41,-0.01,45031.0,-45043.0,82762.0,16834.78,33.0,M-SC,6.63,233.0,-1.00,0.59,5.93,OX40N,NTT,AC
22,DIXON,18931.72,2.51,-8.67,38.04,26.07,67823.0,-16926.0,178295.0,-55.63,42.0,X-MC,8.32,56.0,-0.25,1.27,11.04,X40N,ATH,IT
79,VAIBHAVGBL,521.00,2.20,-25.51,115.82,60.77,157689.0,-46625.0,136150.0,57.02,50.0,H-SC,4.30,159.0,-0.30,0.97,22.61,XR,NTT,JEWELLERY


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,CAMPUS,393.00,-2.05,-12.52,52.36,33.28,76851.0,-21001.0,146774.0,-29.55,35.0,M-SC,2.62,222.0,-0.27,1.05,13.76,XY24,NTT,FOOTWEAR
6,ASIANTILES,137.00,-2.03,-17.45,120.08,81.67,92989.0,-16371.0,77439.0,6955.56,42.0,L-SC,12.98,271.0,-0.18,0.55,49.96,XR,NTT,CERAMICS
67,SONACOMS,806.63,-1.34,-15.94,66.01,39.54,56130.0,-16127.0,85032.0,-34.23,48.0,M-SC,4.63,220.0,-0.29,0.61,19.90,AR,ATH,AUTO
16,CERA,9475.00,-1.22,-32.08,82.40,23.89,98447.0,-56429.0,119474.0,-34.37,28.0,H-SC,10.30,157.0,-0.57,0.85,3.18,OX40N,NTT,CERAMICS
80,VALIANTORG,838.00,-1.18,-43.65,209.05,74.15,192725.0,-71414.0,92191.0,-265.94,47.0,H-SC,22.23,149.0,-0.37,0.66,18.17,XR,NTT,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,398.61,0.59,-0.51,22.67,22.05,22100.0,-495.0,97485.0,-14.18,59.0,H-SC,7.18,164.0,-0.02,0.7,41.7,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.17,-5.16,119.22,107.91,166939.0,-7618.0,140026.0,-22.98,46.0,M-SC,12.64,216.0,-0.05,1.00,4.41,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,1.03,-11.45,12.95,0.02,20018.0,-19981.0,154578.0,135.49,55.0,M-SC,15.39,234.0,-1.00,1.10,34.46,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.17,-16.59,20.00,0.09,16811.0,-16721.0,84056.0,95.09,44.0,M-SC,2.37,253.0,-0.99,0.60,10.43,OX40N,NTT,DURABLES
66,SIS,528.00,1.88,-23.99,59.54,21.26,50247.0,-26640.0,84392.0,1991.19,51.0,H-SC,4.84,166.0,-0.53,0.60,14.02,OX40N,NTT,MISC
47,KANSAINER,340.00,1.31,-24.43,50.16,13.47,102220.0,-65880.0,203787.0,-68.90,38.0,H-SC,4.74,158.0,-0.64,1.45,3.62,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-0.01,6.87,61.19,72.27,110813.0,11648.0,181096.0,-8.35,68.0,M-LC,2.21,99.0,0.11,1.29,13.47,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.01,6.87,61.19,72.27,110813.0,11648.0,181096.0,-8.35,68.0,M-LC,2.21,99.0,0.11,1.29,13.47,XR,NTT,IT
50,MASFIN,398.61,0.59,-0.51,22.67,22.05,22100.0,-495.0,97485.0,-14.18,59.0,H-SC,7.18,164.0,-0.02,0.70,41.70,XR,ATH,FINANCE
38,INDIAMART,4810.62,1.68,-2.47,111.97,106.72,134682.0,-3052.0,120284.0,-51.77,39.0,H-SC,2.66,139.0,-0.02,0.86,20.32,AR,ATH,MISC
25,FINCABLES,1641.55,1.17,-5.16,119.22,107.91,166939.0,-7618.0,140026.0,-22.98,46.0,M-SC,12.64,216.0,-0.05,1.00,4.41,OX40N,ATH,CABLES
37,IEX,219.00,-0.08,-2.90,53.06,48.63,103021.0,-5794.0,194160.0,-35.29,52.0,H-SC,14.03,136.0,-0.06,1.38,8.32,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-1.10,-18.98,91.90,55.47,181139.0,-46181.0,197105.0,-28.25,22.0,X-MC,12.50,64.0,-0.25,1.41,0.99,X40N,ATH,FINANCE
75,TMPV,600.00,-0.27,-28.98,73.16,22.98,117371.0,-65468.0,160430.0,-26.74,30.0,X-LC,6.47,3.0,-0.56,1.14,0.90,XY24,NTT,AUTO
53,PAGEIND,51605.07,-0.43,-8.72,40.35,28.11,74183.0,-17560.0,183850.0,-33.88,31.0,X-MC,12.66,55.0,-0.24,1.31,0.20,X40,ATH,APPARELS
23,DMART,5391.45,-0.56,-8.14,41.09,29.60,94210.0,-20322.0,229278.0,-21.67,32.0,X-LC,5.49,19.0,-0.22,1.64,12.26,X40N,ATH,FMCG
2,ABBOTINDIA,35195.00,-0.41,-6.42,26.17,18.07,43801.0,-11485.0,167370.0,-19.83,32.0,X-MC,5.48,63.0,-0.26,1.19,10.91,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,0.16,-0.89,25.59,24.48,63876.0,-2244.0,249612.0,-81.09,45.0,X-SC,0.69,86.0,-0.04,1.78,22.04,X40N,NTT,MISC
43,ITC,452.0,0.47,-1.47,12.44,10.78,29455.0,-3540.0,236778.0,-41.15,46.0,X-LC,1.91,5.0,-0.12,1.69,3.69,X40,NTT,FMCG
36,ICICIPRULI,790.0,-0.22,6.92,22.26,30.73,46890.0,13640.0,210645.0,-18.05,63.0,X-MC,2.13,75.0,0.29,1.50,20.58,X40,ATH,INSURANCE
33,HINDUNILVR,2922.0,1.36,-10.87,27.53,13.67,63079.0,-27933.0,229130.0,-17.61,33.0,X-LC,2.18,14.0,-0.44,1.63,8.17,XY25,NTT,FMCG
3,ACC,3906.0,0.10,-25.47,120.24,64.15,213246.0,-60601.0,177350.0,-56.31,34.0,X-SC,2.43,82.0,-0.28,1.27,0.10,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,0.10,-25.47,120.24,64.15,213246.0,-60601.0,177350.0,-56.31,34.0,X-SC,2.43,82.0,-0.28,1.27,0.10,XY24,BTT,CEMENT
53,PAGEIND,51605.07,-0.43,-8.72,40.35,28.11,74183.0,-17560.0,183850.0,-33.88,31.0,X-MC,12.66,55.0,-0.24,1.31,0.20,X40,ATH,APPARELS
75,TMPV,600.00,-0.27,-28.98,73.16,22.98,117371.0,-65468.0,160430.0,-26.74,30.0,X-LC,6.47,3.0,-0.56,1.14,0.90,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-1.10,-18.98,91.90,55.47,181139.0,-46181.0,197105.0,-28.25,22.0,X-MC,12.50,64.0,-0.25,1.41,0.99,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.04,-38.08,118.36,35.20,94300.0,-48998.0,79672.0,4.40,34.0,X-SC,15.98,92.0,-0.52,0.57,1.20,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.11,-29.34,102.16,42.84,46925.0,-19073.0,45933.0,-53.72,43.0,X-SC,35.91,83.0,-0.41,0.33,3.92,XY24,NTT,MISC
59,RELAXO,1176.00,-0.36,-47.73,191.38,52.31,145213.0,-69283.0,75877.0,-44.34,41.0,X-SC,6.72,91.0,-0.48,0.54,1.57,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.04,-38.08,118.36,35.20,94300.0,-48998.0,79672.0,4.40,34.0,X-SC,15.98,92.0,-0.52,0.57,1.20,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.69,-4.44,29.84,24.08,36201.0,-5633.0,121317.0,-12.23,33.0,X-SC,5.23,89.0,-0.16,0.87,14.31,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.38,-16.44,71.51,43.32,97325.0,-26772.0,136100.0,-28.18,37.0,X-SC,4.63,90.0,-0.28,0.97,4.50,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,0.13,-11.33,36.14,20.71,121202.0,-42868.0,335369.0,-23.62,64.0,X-LC,4.66,1.0,-0.35,2.39,12.06,X40,ATH,IT
41,INFY,1972.00,0.41,9.25,22.88,34.24,78209.0,28934.0,341822.0,-14.93,64.0,X-LC,3.58,2.0,0.37,2.44,18.30,X40,NTT,IT
75,TMPV,600.00,-0.27,-28.98,73.16,22.98,117371.0,-65468.0,160430.0,-26.74,30.0,X-LC,6.47,3.0,-0.56,1.14,0.90,XY24,NTT,AUTO
81,VBL,671.64,0.30,-3.05,39.94,35.67,122299.0,-9634.0,306208.0,-14.52,58.0,X-LC,4.62,4.0,-0.08,2.18,10.38,X40N,ATH,FMCG
43,ITC,452.00,0.47,-1.47,12.44,10.78,29455.0,-3540.0,236778.0,-41.15,46.0,X-LC,1.91,5.0,-0.12,1.69,3.69,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.34,-37.32,112.01,32.89,54205.0,-28814.0,48393.0,-697.80,56.0,L-MC,5.97,259.0,-0.53,0.35,33.27,XR,NTT,BANKS
6,ASIANTILES,137.00,-2.03,-17.45,120.08,81.67,92989.0,-16371.0,77439.0,6955.56,42.0,L-SC,12.98,271.0,-0.18,0.55,49.96,XR,NTT,CERAMICS
50,MASFIN,398.61,0.59,-0.51,22.67,22.05,22100.0,-495.0,97485.0,-14.18,59.0,H-SC,7.18,164.0,-0.02,0.70,41.70,XR,ATH,FINANCE
13,BSOFT,831.70,0.75,-21.52,92.12,50.79,101691.0,-30263.0,110390.0,0.50,68.0,H-SC,6.08,151.0,-0.30,0.79,28.53,XR,ATH,IT
39,INDIGOPNTS,1408.00,1.03,-11.45,12.95,0.02,20018.0,-19981.0,154578.0,135.49,55.0,M-SC,15.39,234.0,-1.00,1.10,34.46,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.75,-21.52,92.12,50.79,101691.0,-30263.0,110390.0,0.50,68.0,H-SC,6.08,151.0,-0.30,0.79,28.53,XR,ATH,IT
84,WIPRO,420.00,-0.01,6.87,61.19,72.27,110813.0,11648.0,181096.0,-8.35,68.0,M-LC,2.21,99.0,0.11,1.29,13.47,XR,NTT,IT
41,INFY,1972.00,0.41,9.25,22.88,34.24,78209.0,28934.0,341822.0,-14.93,64.0,X-LC,3.58,2.0,0.37,2.44,18.30,X40,NTT,IT
32,HCLTECH,1922.01,0.77,9.42,14.00,24.73,37058.0,22778.0,264702.0,14.46,67.0,X-LC,6.73,7.0,0.61,1.89,26.09,X40,ATH,IT
1,ABB,7934.00,-0.06,-3.22,50.41,45.57,127638.0,-8419.0,253200.0,-37.78,59.0,H-MC,4.58,120.0,-0.07,1.81,7.74,AR,NTT,ELECTRICAL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.33
1,25,44.90
2,50,76.61


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.98
MC    30.91
LC    24.07
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.78
X40      22.94
X40N     13.03
XR        9.82
XY25      9.07
AR        9.04
OX40N     7.53
SR        0.98
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.40
H-SC    23.88
X-LC    20.65
M-SC    11.64
X-SC     8.05
H-MC     4.82
L-SC     1.41
M-MC     1.34
M-LC     1.29
H-LC     1.12
L-LC     1.01
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.32,-7.37,41.89
IT,13.02,-15.13,73.03
FINANCE,9.56,-18.97,70.71
MISC,7.31,-26.48,78.22
ELECTRICAL,6.02,-11.29,52.06
PAINTS,5.85,-12.59,29.31
INSURANCE,4.77,-1.28,35.93
PHARMA,4.11,-1.75,33.78
AUTO,2.82,-32.75,78.78


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3165124.0,21
AR,1305364.0,10
XR,1286547.0,13
X40,1005845.0,14
X40N,858404.0,9
OX40N,734830.0,10
XY25,368480.0,6
SR,281842.0,2
MH,76098.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3629424.0,25
M-SC,1424516.0,15
X-MC,1402978.0,16
X-LC,831580.0,11
X-SC,791784.0,8
H-MC,402581.0,3
L-SC,263693.0,3
M-LC,110813.0,1
H-LC,76476.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1263099.0      6
           AR         894171.0      5
M-SC       XY24       819629.0      6
H-SC       XR         771012.0      7
X-MC       X40        489733.0      7
           XY24       399456.0      3
H-SC       OX40N      343202.0      4
X-MC       X40N       335481.0      4
X-LC       X40        327114.0      5
X-SC       X40N       306414.0      3
M-SC       OX40N      305768.0      5
X-SC       XY24       296372.0      3
H-SC       SR         281842.0      2
X-LC       X40N       216509.0      2
H-MC       AR         208282.0      2
           XY24       194299.0      1
X-LC       XY24       192269.0      2
X-SC       X40        188998.0      2
X-MC       XY25       178308.0      2
L-SC       XR         177833.0      2
M-SC       XR         172684.0      2
           AR         126435.0      2
M-LC       XR         110813.0      1
X-LC       XY25        95688.0      2
L-SC       OX40N       85860.0      1
H-LC       AR          76476.0      1
H-SC       MH          76098.0      1
M-MC       XY25        57230.0      1
L-MC       XR          54205.0      1
L-LC       XY25        37254.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
